In [1]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from scipy.stats import uniform, randint
import numpy as np
import joblib
from sklearn.preprocessing import LabelEncoder
import anndata as ad
import pandas as pd
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import xgboost as xgb
import scanpy as sc

# Buscar os melhores parâmetros

In [2]:
import pickle  # or use joblib

adata=ad.read_h5ad('/home/local.hcpa.ufrgs.br/tkruger/V02_Glioblastoma_atlas/adatas/adata.h5ad')

with open("final_xgb_model_20250504_025832.pkl", "rb") as f:
    model = pickle.load(f)

importance = model.feature_importances_
feature_names = adata.var_names

importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importance
}).sort_values(by='Importance', ascending=False)

importance_df

adatab=ad.read_h5ad('/home/local.hcpa.ufrgs.br/tkruger/V02_Glioblastoma_atlas/adatas/adata_5.h5ad')

#Make row names unique
adata.obs_names_make_unique()
adatab.obs_names_make_unique()

#Find intersections between adatas
match = np.intersect1d(adata.obs_names, adatab.obs_names)

#Keep adata matrix with obs that are in adatab
adata = adata[match]

#Map classes from adatab to adata
adata.obs['broad_cell_type'] = adata.obs.index.map(adatab.obs['broad_cell_type'])

/home/local.hcpa.ufrgs.br/tkruger/Install/conda/miniconda3/envs/ML/lib/python3.9/site-packages/anndata/_core/anndata.py:1897: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/tmp/ipykernel_292326/254197543.py:31: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['broad_cell_type'] = adata.obs.index.map(adatab.obs['broad_cell_type'])


# Remove 98,8% of features

In [3]:
remove = importance_df.tail(13010)['Feature']
adata = adata[:, ~adata.var_names.isin(remove)]
X = adata.X
if not isinstance(X, np.ndarray):
    X_array = X.toarray()
else:
    X_array = X
y = adata.obs['broad_cell_type']
df = pd.DataFrame({'label': y.values})
np.save('main_X_50.npy', X_array)
np.save('main_Y_50.npy', y)
df.to_csv('main_df_50.csv')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_dense = X_train.toarray()
X_test_dense = X_test.toarray()
row_sums_train = X_train_dense.sum(axis=1, keepdims=True)
row_sums_test = X_test_dense.sum(axis=1, keepdims=True)
X_train_normalized = X_train_dense / row_sums_train * 1000
X_test_normalized = X_test_dense / row_sums_test * 1000
X_train_log1p = np.log1p(X_train_normalized)
X_test_log1p = np.log1p(X_test_normalized)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_log1p)
X_test_scaled = scaler.fit_transform(X_test_log1p)
np.save('feature_reduction/01_X_train.npy',X_train_scaled)
np.save('feature_reduction/01_y_train.npy',y_train)
np.save('feature_reduction/01_X_test.npy',X_test_scaled)
np.save('feature_reduction/01_y_test.npy',y_test)

# Search for best hyperparameters

In [ ]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from scipy.stats import uniform, randint
import numpy as np
import joblib
from sklearn.preprocessing import LabelEncoder

#Load train data
train_data_subset = np.load('feature_reduction/01_X_train.npy')
train_labels_subset = np.load('feature_reduction/01_y_train.npy', allow_pickle=True)

#Encode labels
label_encoder = LabelEncoder()

train_labels_encoded = label_encoder.fit_transform(train_labels_subset)

#Define model
model = XGBClassifier(use_label_encoder=False, eval_metric='logloss',n_jobs=1)

# Define the parameter distributions
param_distributions = {
    'n_estimators': randint(100, 1000),
    'max_depth': randint(3, 10),
    'learning_rate': uniform(0.01, 0.3),
    'subsample':uniform(0.5,1)
    'colsample_bytree': uniform(0.5, 1),
    'gamma': uniform(0, 5),
    'min_child_weight': randint(1, 10),
    'reg_alpha': uniform(0, 1),
    'reg_lambda': uniform(0, 1)
}

# Random search
random_search = RandomizedSearchCV(
    model,
    param_distributions,
    n_iter=50,
    cv=3,
    random_state=42,
    n_jobs=1,
    verbose=2
)

random_search.fit(train_data_subset, train_labels_encoded)

joblib.dump(random_search.best_estimator_, 'reduced_best_xgb_model.pkl')

import json

# Get the best hyperparameters
best_params = random_search.best_params_

# Save them to a JSON file
with open('reduced_best_xgb_params.json', 'w') as f:
    json.dump(best_params, f, indent=4)